In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=100,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

In [9]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them with each step containing just one line explanation.:
{tools}
''')

In [4]:
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [6]:
string_parser = StrOutputParser()

In [10]:
chain_tools = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()})
chain_strategy = chat_template_strategy | chat | string_parser

In [11]:
print(chain_tools.invoke({'job title':'data scientist'}))

{'tools': '1. Python\n2. Jupyter Notebook\n3. pandas\n4. matplotlib\n5. scikit-learn'}


In [12]:
chain_combined = chain_tools | chain_strategy

In [13]:
print(chain_combined.invoke({'job title':'data scientist'}))

Here's a strategy for effectively learning and mastering each tool:

**Step 1: Python**
Start by learning the basics of Python programming, focusing on syntax, data types, control structures, functions, and modules.

**Step 2: Jupyter Notebook**
 Familiarize yourself with Jupyter Notebook by creating a new notebook, understanding its interface, and experimenting with basic commands and shortcuts.

**Step 3: pandas**
Master pandas by starting with basic data manipulation and analysis tasks, such as reading


In [14]:
chain_long = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()} | 
              chat_template_strategy | chat | string_parser)

In [15]:
pip install grandalf

  Using cached grandalf-0.8-py3-none-any.whl.metadata (1.7 kB)
Using cached grandalf-0.8-py3-none-any.whl (41 kB)
Note: you may need to restart the kernel to use updated packages.


In [16]:
chain_long.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
   +-----------------+     
   | ChatHuggingFace |     
   +-----------------+     
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       